In [4]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [8]:
calculator = "https://www.selflessclothes.com/blog/sustainability-calculator/?"

In [46]:
goal = "https://www.selflessclothes.com/blog/sustainability-calculator/?material=COTTON&material=POLYESTER&percentage=50&percentage=50"

In [1]:
item = {'item': 'Linen Blend Belted Sleeveless Lapel Dress (JW Anderson)', 'silk': 53, 'linen': 33, 'cotton': 14}

In [6]:
material_params = ["COTTON", "RECYCLED_COTTON", "ORGANIC_COTTON", "POLYESTER", "RECYCLED_POLYESTER", "NYLON", "RECYCLED_NYLON", "ACRYLIC", "SPANDEX", "FLAX", "LINEN", "HEMP", "CUPRO", "LYOCELL", "TENCEL_LYOCELL_LENZING", "REFIBRA_TENCEL_LYOCELL_LENZING", "MODAL", "TENCEL_MODAL_LENZING", "VISCOSE", "VISCOSE_BAMBOO", "VISCOSE_ASIA_LENZING", "VISCOSE_EU_LENZING", "SILK", "ALPACA", "WOOL", "RECYCLED_WOOL", "CASHMERE", "RECYCLED_CASHMERE"]

In [2]:
materials_and_percents = {key: value for key, value in item.items() if key != 'item'}
materials_and_percents

{'silk': 53, 'linen': 33, 'cotton': 14}

In [9]:
goal_params = []
for material, percent in materials_and_percents.items():
    if material.upper() in material_params:
        goal_params.extend([f"material={material.upper()}", f"percentage={percent}"])

# Reorder the goal_params list to have materials first
reordered_params = [goal_params[i] for i in range(0, len(goal_params), 2)] + [goal_params[i] for i in range(1, len(goal_params), 2)]

goal_url = calculator + "&".join(reordered_params)
print(goal_url)

https://www.selflessclothes.com/blog/sustainability-calculator/?material=SILK&material=LINEN&material=COTTON&percentage=53&percentage=33&percentage=14


In [12]:
def get_score_using_selenium(goal_url):
    # Configure Chrome options for headless mode
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Run in headless mode

    # Initialize Chrome WebDriver with configured options
    driver = webdriver.Chrome(options=chrome_options)

    try:
        # Load the page
        driver.get(goal_url)

        # Wait for the score element to be present
        score_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "ml-2"))
        )

        # Extract the score text
        score = score_element.text.strip()

        return score
    except Exception as e:
        print("Error:", e)
        return None
    finally:
        # Close the WebDriver
        driver.quit()

# Example usage:
# goal_url = "https://www.selflessclothes.com/blog/sustainability-calculator/?material=COTTON&material=POLYESTER&percentage=50&percentage=50"
score = get_score_using_selenium(goal_url)
if score:
    print("Score:", score)
else:
    print("Score not found")

Score: (0.2)
